load libraries

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

Using TensorFlow backend.


In [2]:
dir = "E:\DL_data\distracted_driver_detection\imgs"

model_image_size = (224, 224)
fine_tune_layer = 22
final_layer = 24
visual_layer = 21
batch_size = 64

def lambda_func(x):
    x /= 255.
    x -= 0.5
    x *= 2
    return x

# 加载数据集

load train data

In [3]:
train_gen = ImageDataGenerator(
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
    rotation_range=10.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.1,
)
gen = ImageDataGenerator(
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
)

train_generator = train_gen.flow_from_directory(os.path.join(dir, 'train'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical")
print("subdior to train type {}".format(train_generator.class_indices))
valid_generator = gen.flow_from_directory(os.path.join(dir, 'valid'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical")
print("subdior to valid type {}".format(valid_generator.class_indices))

F:\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:348: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 22424 images belonging to 10 classes.
subdior to train type {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}
Found 0 images belonging to 0 classes.
subdior to valid type {}


# 构建模型

https://github.com/fchollet/keras/blob/master/keras/applications/vgg19.py


In [4]:
input_tensor = Input((*model_image_size, 3))
x = input_tensor
# if lambda_func:
#     x = Lambda(lambda_func)(x)

base_model = MobileNetV2(input_tensor=Input((*model_image_size, 3)), weights='imagenet', include_top=False)

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(base_model.input, x)

print("total layer count {}".format(len(base_model.layers)))

for i in range(fine_tune_layer):
    model.layers[i].trainable = False

F:\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '



9412608/9406464 [==============================] - 37s 4us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
total layer count 155


# 训练模型

In [5]:
print("train_generator.samples = {}".format(train_generator.samples))
print("valid_generator.samples = {}".format(valid_generator.samples))
steps_train_sample = train_generator.samples // 128 + 1
steps_valid_sample = valid_generator.samples // 128 + 1

train_generator.samples = 22424
valid_generator.samples = 0


In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=10, validation_data=valid_generator, validation_steps=steps_valid_sample)

model.save("models/MobileNetV2-imagenet-finetune{}-adam.h5".format(fine_tune_layer))
print("model saved!")


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10


F:\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
F:\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


176/176 [==============================] - 191s 1s/step - loss: 0.3310 - acc: 0.8975
Epoch 2/10
176/176 [==============================] - 189s 1s/step - loss: 0.1056 - acc: 0.9681
Epoch 3/10
176/176 [==============================] - 117s 665ms/step - loss: 0.0714 - acc: 0.9810
Epoch 4/10
176/176 [==============================] - 117s 664ms/step - loss: 0.0641 - acc: 0.9820
Epoch 5/10
176/176 [==============================] - 118s 669ms/step - loss: 0.0554 - acc: 0.9826
Epoch 6/10
176/176 [==============================] - 117s 666ms/step - loss: 0.0526 - acc: 0.9856
Epoch 7/10
176/176 [==============================] - 118s 668ms/step - loss: 0.0364 - acc: 0.9898
Epoch 8/10
176/176 [==============================] - 117s 666ms/step - loss: 0.0418 - acc: 0.9880
Epoch 9/10
176/176 [==============================] - 118s 669ms/step - loss: 0.0319 - acc: 0.9907
Epoch 10/10
176/176 [==============================] - 117s 667ms/step - loss: 0.0376 - acc: 0.9884
model saved!


In [8]:
model.compile(optimizer=RMSprop(lr=1*0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=10, validation_data=valid_generator, validation_steps=steps_valid_sample)

model.save("models/MobileNetV2-imagenet-finetune{}.h5".format(fine_tune_layer))
print("model saved!")

Epoch 1/10
176/176 [==============================] - 119s 677ms/step - loss: 0.0278 - acc: 0.9911
Epoch 2/10
176/176 [==============================] - 119s 676ms/step - loss: 0.0221 - acc: 0.9937
Epoch 3/10
176/176 [==============================] - 120s 681ms/step - loss: 0.0171 - acc: 0.9950
Epoch 4/10
176/176 [==============================] - 120s 679ms/step - loss: 0.0110 - acc: 0.9959
Epoch 5/10
176/176 [==============================] - 120s 683ms/step - loss: 0.0099 - acc: 0.9974
Epoch 6/10
176/176 [==============================] - 120s 680ms/step - loss: 0.0108 - acc: 0.9966
Epoch 7/10
176/176 [==============================] - 120s 683ms/step - loss: 0.0069 - acc: 0.9978
Epoch 8/10
176/176 [==============================] - 120s 680ms/step - loss: 0.0073 - acc: 0.9980
Epoch 9/10
176/176 [==============================] - 120s 681ms/step - loss: 0.0055 - acc: 0.9985
Epoch 10/10
176/176 [==============================] - 120s 681ms/step - loss: 0.0059 - acc: 0.9980
model sav

# 可视化模型

https://keras.io/visualization/

In [9]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import *

model = load_model("models/MobileNetV2-imagenet-finetune{}.h5".format(fine_tune_layer))
print("load successed")

#SVG(model_to_dot(model).create(prog='dot', format='svg'))

load successed


# CAM 可视化

http://cnnlocalization.csail.mit.edu/

![](http://cnnlocalization.csail.mit.edu/framework.jpg)

$cam = (P-0.5)*w*output$

* cam: 类激活图 X\*X
* P: 概率
* output: 卷积层的输出 512\*1
* w: 卷积核的权重 X\*X\*512

In [10]:
z = zip([x.name for x in model.layers], range(len(model.layers)))
for k, v in z:
    print("{} - {}".format(k,v))

input_2 - 0
Conv1_pad - 1
Conv1 - 2
bn_Conv1 - 3
Conv1_relu - 4
expanded_conv_depthwise - 5
expanded_conv_depthwise_BN - 6
expanded_conv_depthwise_relu - 7
expanded_conv_project - 8
expanded_conv_project_BN - 9
block_1_expand - 10
block_1_expand_BN - 11
block_1_expand_relu - 12
block_1_pad - 13
block_1_depthwise - 14
block_1_depthwise_BN - 15
block_1_depthwise_relu - 16
block_1_project - 17
block_1_project_BN - 18
block_2_expand - 19
block_2_expand_BN - 20
block_2_expand_relu - 21
block_2_depthwise - 22
block_2_depthwise_BN - 23
block_2_depthwise_relu - 24
block_2_project - 25
block_2_project_BN - 26
block_2_add - 27
block_3_expand - 28
block_3_expand_BN - 29
block_3_expand_relu - 30
block_3_pad - 31
block_3_depthwise - 32
block_3_depthwise_BN - 33
block_3_depthwise_relu - 34
block_3_project - 35
block_3_project_BN - 36
block_4_expand - 37
block_4_expand_BN - 38
block_4_expand_relu - 39
block_4_depthwise - 40
block_4_depthwise_BN - 41
block_4_depthwise_relu - 42
block_4_project - 43
bl

http://docs.opencv.org/trunk/d3/d50/group__imgproc__colormap.html

![](http://docs.opencv.org/trunk/colorscale_jet.jpg)

In [15]:
import matplotlib.pyplot as plt
import random
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

def show_heatmap_image(model_show, weights_show):
    test_dir="./test"
    image_files = glob.glob(os.path.join(test_dir,"*"))
    print(len(image_files))
    
    plt.figure(figsize=(12, 24))
    for i in range(10):
        plt.subplot(5, 2, i+1)
        #img = cv2.imread(image_files[2000*i+113])
        img = cv2.imread(image_files[i])
        img = cv2.resize(img,  (model_image_size[1],model_image_size[0]))
        x = img.copy()
        x.astype(np.float32)
        out, predictions = model_show.predict(np.expand_dims(x, axis=0))
        predictions = predictions[0]
        out = out[0]
        
        max_idx = np.argmax(predictions)
        prediction = predictions[max_idx]

        status = ["safe driving",  " texting - right",  "phone - right",  "texting - left",  "phone - left",  
                  "operation radio", "drinking", "reaching behind", "hair and makeup", "talking"]

        plt.title('c%d |%s| %.2f%%' % (max_idx , status[max_idx], prediction*100))
        
        #以上开始都容易理解问题在于下面的就不知道该如何是好了
        
    
    
        cam = (prediction - 0.5) * np.matmul(out, weights_show)
        cam = cam[:,:,max_idx]
        cam -= cam.min()
        cam /= cam.max()
        cam -= 0.2
        cam /= 0.8

        cam = cv2.resize(cam, (model_image_size[1],model_image_size[0]))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam <= 0.2)] = 0

        out = cv2.addWeighted(img, 0.8, heatmap, 0.4, 0)

        plt.axis('off')
        plt.imshow(out[:,:,::-1])
print("done")

done


In [16]:
weights = model.layers[final_layer].get_weights()[0]
layer_output = model.layers[visual_layer].output
model2 = Model(model.input, [layer_output, model.output])
print("layer_output {0}".format(layer_output))
print("weights shape {0}".format(weights.shape))
show_heatmap_image(model2, weights)

IndexError: list index out of range

In [15]:
def gen_kaggle_csv(model,  model_image_size, csv_name):
    #dir = "/ext/Data/distracted_driver_detection/"
    dir = "D:\Tensorflow\DMS\mlnd_distracted_driver_detection-master\data\distracted_driver_detection\imgs"

    gen = ImageDataGenerator()
    test_generator = gen.flow_from_directory(dir + "test/",  model_image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
#     s = test_generator.__dict__
#     del s['filenames']
#     print(s)
    y_pred = model.predict_generator(test_generator,  steps=test_generator.samples//batch_size+1,  verbose=1)
    print("y_pred shape {}".format(y_pred.shape))
    y_pred = y_pred.clip(min=0.005, max=0.995)
    print(y_pred[:3])

    l = list()
    for i, fname in enumerate(test_generator.filenames):
        name = fname[fname.rfind('/')+1:]
        l.append( [name, *y_pred[i]] )

    l = np.array(l)
    data = {'img': l[:,0]}
    for i in range(10):
        data["c%d"%i] = l[:,i+1]
    df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
    df.head(10)
    df = df.sort_values(by='img')
    df.to_csv(csv_name, index=None, float_format='%.3f')
    print("csv saved")

print("done")

done


In [16]:
gen_kaggle_csv(model,  model_image_size, 'csv/MobileNetV2-imagenet-finetune{}-pred.csv'.format(fine_tune_layer))

Found 79726 images belonging to 1 classes.
1246/1246 [==============================] - 314s     